Prova di Soundstream, è necessario usare pip install soundstream e pip install torchaudio

In [2]:
import torchaudio

from soundstream import from_pretrained, load

c:\Users\fabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
waveform = load('canterburytales_09_chaucer_64kb_0000.flac')


In [4]:
audio_codec = from_pretrained()  # Pretrained model from Hugging Face

In [5]:
quantized = audio_codec(waveform, mode='encode') #Those are the tokens
print(quantized)

tensor([[[ 2.2459e-01, -2.2389e-02,  1.5525e-01,  ...,  5.6992e-01,
          -3.2122e+00,  4.4403e-01],
         [-8.3809e-03, -7.3805e-02, -2.1592e-01,  ..., -3.8940e-03,
          -4.0144e+00,  4.8721e-01],
         [-1.9654e-01,  2.7941e-02,  1.0666e-01,  ..., -4.9368e-01,
          -7.8120e-01, -2.8934e-01],
         ...,
         [-2.1161e-01,  4.3835e-02,  7.8821e-01,  ..., -2.5401e-01,
           1.1656e+00, -1.6173e-01],
         [ 7.2735e-01,  8.1265e-02, -4.5434e-01,  ...,  2.4940e-01,
           2.7430e-01,  1.6015e+00],
         [ 3.5554e-01, -3.3391e-01, -4.8876e-01,  ..., -8.8031e-02,
          -1.5547e+00,  1.9502e-01]]], grad_fn=<AddBackward0>)


In [6]:
recovered = audio_codec(quantized, mode='decode') #From tokens to waveform tensor (3D, we need 2D)

In [7]:
print(recovered[0])

waveform = recovered[0] #3D to 2D
waveform = waveform.cpu().detach() #Cuts out the gradient part since torchaudio.save doesn't work with it

tensor([[ 0.0038,  0.0010, -0.0014,  ..., -0.0027, -0.0032, -0.0122]],
       grad_fn=<SelectBackward0>)


In [18]:
print("Original Waveform Shape:", waveform.shape)
print("Quantized Tokens Shape:", quantized.shape)
print("Recovered Waveform Shape:", recovered.shape)

Original Waveform Shape: torch.Size([1, 936800])
Quantized Tokens Shape: torch.Size([1, 4684, 256])
Recovered Waveform Shape: torch.Size([1, 1, 936800])


In [8]:
torchaudio.save('out.wav', waveform, 16000)

In [9]:
from IPython.display import Audio
Audio(waveform.numpy()[0], rate=16000)